<a href="https://colab.research.google.com/github/anzardraboo/Computer-Vision/blob/master/Damage_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/matterport/Mask_RCNN.git

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


In [2]:
!unzip /content/customdataset.zip

Archive:  /content/customdataset.zip
replace customdataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: customdataset/.DS_Store  
replace __MACOSX/customdataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace __MACOSX/customdataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/customdataset/._.DS_Store  
  inflating: customdataset/train/image63.jpg  
  inflating: __MACOSX/customdataset/train/._image63.jpg  
  inflating: customdataset/train/image62.jpg  
  inflating: __MACOSX/customdataset/train/._image62.jpg  
  inflating: customdataset/train/image18.jpeg  
  inflating: __MACOSX/customdataset/train/._image18.jpeg  
  inflating: customdataset/train/image60.jpg  
  inflating: __MACOSX/customdataset/train/._image60.jpg  
  inflating: customdataset/train/image48.jpg  
  inflating: __MACOSX/customdataset/train/._image48.jpg  
  inflating: customdataset/train/image49.jpg  
  inflating: __MACOSX/customda

In [3]:
!ls

customdataset  customdataset.zip  custom.py  __MACOSX  Mask_RCNN  sample_data


In [0]:
!mv custom.py Mask_RCNN/

In [5]:
%cd Mask_RCNN/

/content/Mask_RCNN


In [6]:
!mkdir train_logs

mkdir: cannot create directory ‘train_logs’: File exists


In [7]:
!python3 custom.py train --dataset=/content/customdataset --weights=coco --logs=/content/Mask_RCNN/train_logs/

Using TensorFlow backend.
Weights:  coco
Dataset:  /content/customdataset
Logs:  /content/Mask_RCNN/train_logs/

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_l

In [8]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
# Root directory of the project
ROOT_DIR = os.getcwd()
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import custom
%matplotlib inline
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "train_logs/damage20191006T0907")
custom_WEIGHTS_PATH = "mask_rcnn_damage_0004.h5"  # TODO: update this path
config = custom.CustomConfig()
custom_DIR = '/content/customdataset'
class InferenceConfig(config.__class__):
   # Run detection on one image at a time
   GPU_COUNT = 1
   IMAGES_PER_GPU = 1
config = InferenceConfig()
config.display()
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"
def get_ax(rows=1, cols=1, size=16):
   """Return a Matplotlib Axes array to be used in
   all visualizations in the notebook. Provide a
   central point to control graph sizes.
   Adjust the size attribute to control how big to render images
   """
   _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
   return ax
# Load validation dataset
dataset = custom.CustomDataset()
dataset.load_custom(custom_DIR, "val")
# Must call before using the dataset
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))
# Create model in inference mode
with tf.device(DEVICE):
   model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                             config=config)
# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(MODEL_DIR+'/'+custom_WEIGHTS_PATH, by_name=True)
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
   modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,
                                      dataset.image_reference(image_id)))
# Run object detection
results = model.detect([image], verbose=1)
# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                           dataset.class_names, r['scores'], ax=ax,
                           title="Predictions")


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

Using TensorFlow backend.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights  mask_rcnn_damage_0004.h5


OSError: ignored